In [8]:
import numpy as np
import os
import nibabel as nib

from scipy import ndimage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [43]:
def create_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 3))
    print(inputs.shape)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="softmax")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    print('done')
    return model

def deserialize(serialized_example):
    feature_map = {
        'x_shape': tf.io.FixedLenFeature([3], dtype=tf.int64),
        'x': tf.io.FixedLenFeature([], dtype=tf.string),
        'y': tf.io.FixedLenFeature([], dtype=tf.string)
    }

    example = tf.io.parse_single_example(serialized_example, feature_map)

    x_shape = example['x_shape']
    x_bytes = example['x']
    y_bytes = example['y']
    
    x = tf.io.decode_raw(x_bytes, out_type=tf.float64)
    x = tf.reshape(x, x_shape)
    x = tf.expand_dims(x, axis=-1)
    x = tf.repeat(x, 3, axis=-1)

    x = tf.cast(x, tf.float32)
    y = tf.io.parse_tensor(y_bytes, out_type=tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y

In [25]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')



In [32]:
for record in train_data:
    x, y = deserialize(record)
    print(x.shape)
    print(y)
    print(x)
    break


KeyboardInterrupt



In [52]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')
train_data = train_data.map(deserialize)
train_data = train_data.shuffle(10)
train_data = train_data.batch(5)


In [58]:

model = create_model()

initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics='accuracy')

batch_size = 1
epochs = 2
validation_split = 0.1

with tf.device("CPU"):
    model.fit(
        train_data,
        epochs=epochs,
        verbose=2
    )



(None, 128, 128, 64, 3)
done
Epoch 1/2


ValueError: in user code:

    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:862 train_function  *
        return step_function(self, iterator)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:852 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorf